# cia-data-loader

## Objetivos del sistema

✅ Leer cualquier fichero presente en el directorio file.

✅ Generar los dataframe dinamicamente

❌ Identificar las claves primarias de cada dataframe

❌ Identificar las claves forneas de cada dataframe

❌ Crear el esquema en Progress correspondiente a los ficheros incorporados.

❌ Cargar los datos en esquema de base de datos



## 🛠️ Instalación

En este epigrafe realizaremos todos los pasos necesarios para tener el entorno listo para la ejecución

###🔨.Requisitos Previos 

### get_environment_info 

Esta función detecta si el entorno donde se ejecuta el código es:

🐳 Un contenedor Docker

💻 Una máquina virtual (VM)

🖥️ Un sistema físico (bare-metal)

**Dado que es necesario saber que tipo de entorno al principio es mas limpio crear esta función cargar las librerias**

In [31]:
def get_environment_info():
    import os
    import platform
    import subprocess

    def is_docker():
        try:
            if os.path.exists("/.dockerenv"):
                return True
            with open("/proc/1/cgroup", "r") as f:
                if "docker" in f.read() or "containerd" in f.read():
                    return True
        except Exception:
            pass
        return False

    def is_virtual_machine():
        system = platform.system()

        try:
            if system == "Linux":
                try:
                    output = subprocess.check_output(["systemd-detect-virt"], stderr=subprocess.DEVNULL).decode().strip()
                    if output and output != "none":
                        return True, output
                except Exception:
                    pass

                try:
                    with open("/proc/cpuinfo", "r") as f:
                        if "hypervisor" in f.read().lower():
                            return True, "Desconocido (hypervisor detectado)"
                except Exception:
                    pass

                try:
                    output = subprocess.check_output(["dmidecode", "-s", "system-product-name"], stderr=subprocess.DEVNULL).decode().strip().lower()
                    for vm in ["virtualbox", "kvm", "vmware", "microsoft"]:
                        if vm in output:
                            return True, vm.capitalize()
                except Exception:
                    pass

            elif system == "Windows":
                try:
                    ps_script = "Get-CimInstance -ClassName Win32_ComputerSystem | Select-Object -ExpandProperty Model"
                    output = subprocess.check_output(["powershell", "-Command", ps_script], stderr=subprocess.DEVNULL).decode().strip().lower()
                    for vm in ["virtualbox", "kvm", "vmware", "hyper-v", "virtual"]:
                        if vm in output:
                            return True, vm.capitalize()
                except Exception:
                    pass

        except Exception:
            pass

        return False, None

    docker = is_docker()
    vm, vm_type = is_virtual_machine()

    if docker:
        entorno = "docker"
        is_virtual = True
    elif vm:
        entorno = "virtual"
        is_virtual = True
    else:
        entorno = "real"
        is_virtual = False

    return {
        "docker": docker,
        "virtual_machine": vm,
        "hypervisor": vm_type if vm else False,
        "entorno": entorno,
        "is_virtual" : is_virtual
    }

# Ejemplo de uso
env_info = get_environment_info()
print(env_info)

if env_info["entorno"] == "Docker":
    print("🐳 Ejecutándose dentro de un contenedor Docker")
elif env_info["entorno"] == "virtual":
    print(f"💻 Ejecutándose en máquina virtual ({env_info['hypervisor']})")
else:
    print("🖥️ Ejecutándose en máquina física")


{'docker': False, 'virtual_machine': False, 'hypervisor': False, 'entorno': 'real', 'is_virtual': False}
🖥️ Ejecutándose en máquina física


Es necesario saber si el código esta corriendo en la maquina virtual o en local como paso previo ya que el ENV se ejecuta por parte de la máquina virtual en el arranque

In [32]:
if env_info.get("is_virtual") is False:
    !pip install -r cia-data-loader_requirements.txt

Se esta ejecutando en una máquina virtual: is_virtual_machine
  Using cached nbimporter-0.3.4-py3-none-any.whl.metadata (252 bytes)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached nbimporter-0.3.4-py3-none-any.whl (4.9 kB)

  Attempting uninstall: networkx

    Found existing installation: networkx 3.3

    Uninstalling networkx-3.3:

      Successfully uninstalled networkx-3.3

   ---------- ----------------------------- 1/4 [networkx]
   ---------- ----------------------------- 1/4 [networkx]
   ---------- ----------------------------- 1/4 [networkx]
   ---------- ----------------------------- 1/4 [networkx]
   ---------- ---------

  You can safely remove it manually.
  You can safely remove it manually.


### ⛏️ Cadenas previas 

In [33]:
# Al importar el codido py como notebook se genera esta linea que solo es necesario si se llama desde la máquina virtual
if env_info.get("is_virtual") is True:
    %matplotlib inline
    !/usr/bin/env python
    # coding: utf-8


### 🔧Carga de librerias en el entorno

In [34]:
# Variables propias desarolladas por el equipo que ayuda a la construcción del TFM
import library_tfm_v0
# Carga de todas las funciones generadas
from library_tfm_v0 import *

# Permite cargar configuraciones sensibles (como claves API, contraseñas, nombres de usuario, URLs) desde un archivo .env, sin tener que codificarlas directamente en tu script.
from dotenv import load_dotenv


# Carga de CSV
import pandas as pd


# Para generar el gráfico de dependecias
import networkx as nx
import matplotlib.pyplot as plt

# Calculo de localizaciones
import geocoder

# Permite cargar configuraciones sensibles (como claves API, contraseñas, nombres de usuario, URLs) desde un archivo .env, sin tener que codificarlas directamente en tu script.
from dotenv import dotenv_values



### 🔩 Carga de variables externas prefijadas

In [35]:
# Cargar las variables del archivo .env en un diccionario al cargarlo al entorno para 

if (env_info.get("is_virtual") is True):
    env_vars = dotenv_values(".env")
else:
    env_vars = dotenv_values("cia-data-loader.env")
    
# Asignar cada variable como una variable Python, pero con nombre en minúsculas
# De esta forma no tengo que cargarlas manualmente 
for key, value in env_vars.items():
    globals()[key.lower()] = value 
    print(f'{key.lower()}={value}')

postgres_user=admin
postgres_password=secretpassword
postgres_db=mydb
postgres_host=tfm_ai_postgres_container
python_file=files


### 🔩Importación de librerias

## 💙 Main

### Inicialización de variables

En este punto daremos los valores a las variables 

In [39]:
if env_info.get("is_virtual") is False:
    clean_file(r'C:\Users\MX0046001DC5030\files', extensions=['.txt'])   
    dataframes= load_files_in_dataframes(r'C:\Users\MX0046001DC5030\files', sep=',', extensions=['.txt'])
else:
    clean_file(python_file,extensions=['.txt', '.csv'])   
    dataframes, list_id =load_files_in_dataframes(python_file, sep=',', extensions=['.txt', '.csv'])




if dataframes:
	for file_name, df in dataframes.items():
		print(f"\nInformación del DataFrame '{f'df_{file_name}'}':") # Imprime el nombre del DataFrame

		# print(f'Columnas :{globals()[f"df_{file_name}"].columns.tolist()}') #Imprime el nombre de cada columna 


		# print(f'CATA DEL DATAFRAME \n{globals()[f"df_{file_name}"].head(explore)}') #Imprime  un head de la tabla


	# ## Identificar dependencias entre los Df



tempGlobal = globals() ## Para evitar que intervengan las variables de esta celda
relactions = {}  # Define el diccionario relaciones aquí
relactions_M4 = {}
relactions_NO_M4 = {}

# Crear un grafo dirigido
G = nx.DiGraph()
G_NO_M4 = nx.DiGraph()
G_M4 = nx.DiGraph()
###PARA las tablas M4 y No M4 diferencias por la raiz

dependency_dataframes(globals())
tree_dataframes(globals())


# ## Pasos
# ### Cargar los datos 
# ### Función para identicar la boca de metro más cercana dada la localización del que pregunta 
# ### Función para identificar la geolocalización de la persona ✅ 
# 
# 



location_gps();

Procesando archivo: agency.txt
df_agency lista: ['agency_id']
Procesando archivo: calendar.txt
df_calendar lista: ['service_id', 'agency_id']
Procesando archivo: calendar_dates.txt
df_calendar_dates lista: ['service_id', 'agency_id']
Procesando archivo: fare_attributes.txt
df_fare_attributes lista: ['fare_id', 'service_id', 'agency_id']
Procesando archivo: fare_rules.txt
df_fare_rules lista: ['contains_id', 'route_id', 'agency_id', 'service_id', 'origin_id', 'fare_id', 'destination_id']
Procesando archivo: feed_info.txt
df_feed_info lista: ['contains_id', 'route_id', 'agency_id', 'origin_id', 'fare_id', 'service_id', 'destination_id']
Procesando archivo: frequencies.txt
df_frequencies lista: ['contains_id', 'route_id', 'agency_id', 'service_id', 'origin_id', 'fare_id', 'destination_id', 'trip_id']
Procesando archivo: routes.txt
df_routes lista: ['contains_id', 'route_id', 'agency_id', 'origin_id', 'fare_id', 'service_id', 'destination_id', 'trip_id']
Procesando archivo: shapes.txt
df_s

AttributeError: 'tuple' object has no attribute 'items'